In [ ]:
import os
import numpy as np
import subprocess
import pandas as pd
import requests
from urllib.parse import urljoin
import sqlite3
import json
import time
from datetime import datetime, timedelta
import re
from google.colab import drive
import random
import csv

drive.mount('/content/gdrive/', force_remount=True)
os.chdir("/content/gdrive/Shareddrives/ECS 260/final")

Mounted at /content/gdrive/


In [ ]:
!ls

download_history_1.db  download_history_4.db  final_database.db      npm_download_history.db
download_history_2.db  download_history_5.db  FINAL_PACKAGE_DATA.db  npm_names.txt
download_history_3.db  download_history_6.db  final_packages.txt


In [ ]:
total_parts = 6
dbs = [f'download_history_{i + 1}.db' for i in range(total_parts)]
dbs

['download_history_1.db',
 'download_history_2.db',
 'download_history_3.db',
 'download_history_4.db',
 'download_history_5.db',
 'download_history_6.db']

In [ ]:
new_db_path = "download_history.db"

In [ ]:
# New db init
start_date = datetime(2018, 5, 1)
end_date = datetime(2024, 1, 1)
date_range = pd.date_range(start=start_date, end=end_date, freq='MS')

dest_conn = sqlite3.connect(new_db_path)
dest_cursor = dest_conn.cursor()

column_names = ", ".join(f"m_{month.strftime('%Y%m')} INTEGER" for month in date_range)
sql_query = f"""
        CREATE TABLE IF NOT EXISTS download_data (
            package TEXT,
            {column_names},
            average INTEGER
        )
    """

dest_cursor.execute(sql_query)
dest_conn.commit()

In [ ]:
df = pd.read_sql_query("SELECT * FROM download_data", dest_conn)
df

,package,m_201805,m_201806,m_201807,m_201808,m_201809,m_201810,m_201811,m_201812,m_201901,...,m_202305,m_202306,m_202307,m_202308,m_202309,m_202310,m_202311,m_202312,m_202401,average


In [ ]:
# Merging
# iterate through source databases
for source_db in dbs:
  print(f"Merging {source_db}")

  # connect to the current source database
  source_conn = sqlite3.connect(source_db)
  source_cursor = source_conn.cursor()

  # get all rows from the source database
  source_cursor.execute("SELECT * FROM download_data")
  rows = source_cursor.fetchall()

  # get the column names dynamically
  column_names = [description[0] for description in source_cursor.description]

  # generate the parameter placeholders (?, ?, ?, ...)
  placeholders = ', '.join(['?' for _ in column_names])

  # insert rows into the destination database
  dest_cursor.executemany(f"INSERT INTO download_data ({', '.join(column_names)}) VALUES ({placeholders})", rows)
  # commit changes in the destination connection
  dest_conn.commit()

  # commit changes and close the source connection
  source_conn.commit()
  source_conn.close()
dest_conn.close()

Merging download_history_1.db
Merging download_history_2.db
Merging download_history_3.db
Merging download_history_4.db
Merging download_history_5.db
Merging download_history_6.db


In [ ]:
conn = sqlite3.connect(new_db_path)
cursor = conn.cursor()
df = pd.read_sql_query("SELECT * FROM download_data", conn)
print("Total distinct packages present: ", len(df))
df = pd.read_sql_query("SELECT * FROM download_data", conn)
print("Total rows present: ", len(df))
conn.close()
df

Total distinct packages present:  30000
Total rows present:  30000


,package,m_201805,m_201806,m_201807,m_201808,m_201809,m_201810,m_201811,m_201812,m_201901,...,m_202305,m_202306,m_202307,m_202308,m_202309,m_202310,m_202311,m_202312,m_202401,average
0,cpe-parser,45.0,32.0,39.0,27.0,9.0,4.0,39.0,48.0,44.0,...,8.0,10.0,25.0,11.0,31.0,84.0,23.0,6.0,16.0,24.0
1,jbooter-core,21.0,17.0,19.0,17.0,5.0,7.0,19.0,35.0,24.0,...,5.0,8.0,12.0,7.0,5.0,22.0,10.0,11.0,14.0,13.0
2,img2urf,13.0,9.0,17.0,17.0,2.0,12.0,9.0,19.0,19.0,...,7.0,9.0,10.0,7.0,6.0,11.0,11.0,4.0,6.0,10.0
3,ledgercompany-getjson,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,8.0,6.0,29.0,1.0,6.0,7.0,2.0,3.0,3.0
4,gitbook-start-iaas-bbdd-alex-moi,93.0,49.0,96.0,36.0,15.0,52.0,81.0,76.0,93.0,...,17.0,13.0,37.0,17.0,1.0,54.0,36.0,3.0,31.0,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,@test-sdk-lerna/child-package-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,16.0,11.0,6.0,5.0,56.0,7.0,5.0,16.0,8.0
29996,@hyper.fun/carbon-icon-taxi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,9.0,11.0,5.0,9.0,30.0,13.0,14.0,3.0,5.0
29997,@anysphere/file-service-darwin-x64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8820.0,13388.0,7194.0,8610.0,14731.0,764.0
29998,@dynamic-framework/ui-react,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,279.0,515.0,534.0,791.0,311.0,572.0,538.0,51.0


In [ ]:
# adding this table to final database and adding average monthly download column to final database
import sqlite3

main_conn = sqlite3.connect("final_database.db")
main_cursor = main_conn.cursor()

main_cursor.execute("ATTACH DATABASE 'download_history.db' AS download_history")
main_cursor.execute("""
    CREATE TABLE IF NOT EXISTS download_data AS
    SELECT * FROM download_history.download_data WHERE 1 = 0
""")
main_conn.commit()

main_cursor.execute("""
    INSERT INTO download_data
    SELECT * FROM download_history.download_data
""")
main_conn.commit()

main_cursor.execute("ALTER TABLE package_data ADD COLUMN avg_monthly_download INTEGER")
main_conn.commit()

main_cursor.execute("""
    UPDATE package_data
    SET avg_monthly_download = (
        SELECT average
        FROM download_data AS dh
        WHERE dh.package = package_data.package
    )
""")

main_conn.commit()

main_cursor.execute("DETACH DATABASE download_history")
main_conn.commit()

main_conn.close()